# CAI Prompt

In [52]:
import os
import openai
from dotenv import load_dotenv
import concurrent.futures
load_dotenv()

number_prompts = 5

## Generate tasks

In [54]:

prompt = f"Generate a list of {number_prompts} tasks for a customer service assistant involving reading and writing. Write each task on a separate line and number each task."

openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": prompt}
  ]
)

tasks_string = completion.choices[0].message.content
print(tasks_string)

1. Responding to customer inquiries via email or chat by reading and writing clear, helpful messages.
2. Taking notes during customer calls and summarizing the key points in writing for follow-up.
3. Creating and updating customer service documentation, such as FAQs or instructional guides, that require researching and writing accurate information.
4. Editing and proofreading customer-facing materials (e.g., website copy, newsletters) to ensure they are clear, concise, and error-free.
5. Drafting and sending follow-up surveys to customers after interactions with the company, analyzing the data received, and conveying insights in writing to the customer service team.


In [55]:
def getItems(str):
    items = []
    for line in str.splitlines():
        items.append(line[line.find(".") + 2:])
    return items

tasks = getItems(tasks_string)
print(len(tasks))

5


## Generate naive prompts

> **Terminology** - here we're prompting an LLM to generate prompts, which is confusing. We'll use the word "prompt" to refer to the initial input, and "response" to refer to the output

In [57]:
def get_llm_response(task, i):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": f"Here is a task for a customer service assistant:\n\n{task}\n\nWrite a prompt for a large language model (LLM), asking it to carry out this task."
            }
        ],
        max_tokens=1024,
        temperature=0.5,
        stream=False
    )
    str = response['choices'][0]['message']['content']
    print(f"Response {i} received")
    return str

with concurrent.futures.ThreadPoolExecutor() as executor:
    naive_responses = list(executor.map(lambda x: get_llm_response(x[0], x[1]), [(task, i) for i, task in enumerate(tasks)]))

tasksResponsesString = "\n\n---\n\n".join([f"TASK:\n{task}\n\nNAIVE PROMPT:\n{response}" for task, response in zip(tasks, naive_responses)])
print(tasksResponsesString)

Response 3 received
Response 1 received
Response 4 received
Response 0 received
Response 2 received
TASK:
Responding to customer inquiries via email or chat by reading and writing clear, helpful messages.

NAIVE PROMPT:
Please generate responses to customer inquiries via email or chat that are clear, helpful, and informative. The responses should address the customer's questions or concerns and provide solutions or next steps as necessary. Please ensure that the language used is professional and courteous, with a focus on providing excellent customer service.

---

TASK:
Taking notes during customer calls and summarizing the key points in writing for follow-up.

NAIVE PROMPT:
"Please generate a summary of the key points discussed during customer calls and provide a written record for follow-up. Your notes should accurately capture the customer's concerns, questions, and any solutions or next steps discussed during the call."

---

TASK:
Creating and updating customer service documentat

Most of these responses are slightly more detailed rewrites of the corresponding task. This is a good start, but ultimately not very useful and doesn't demonstrate most of the good practices in prompt design.